<a href="https://colab.research.google.com/github/LeeRippon/KilnVisual/blob/main/KilnVisualizationTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tutorial from "*Visualization of Multiscale Ring Formation in a Rotary Kiln*"

**License:**

**Authors:** Lee Rippon, Barry Hirtz, Carl Sheehan, Travis Reinheimer, Philip Loewen and Bhushan Gopaluni

**Last Revision:** March 5, 2021

## Instructions

- This is a Google Colab notebook and it will provide the Python environment for this tutorial. 

- There are code cells and text cells. To run a code cell click on it then hold shift and press enter. Code cells are to be run in order as inputs to later cells are dependent on outputs of prior cells.

- The third part of this tutorial provides instructions for loading your own operating data.

## 1. Import required packages

In [193]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## 2. Generate synthetic data

It is created to resembles the format of data extracted from a process historian data.

In [217]:
dti = pd.date_range("2018-07-01", periods=20000, freq="H")
t = np.arange(0,len(dti))
# Mean temperatures in deg F
y = [1410, 1379, 1360,  1327, 1232, 1188, 1127, 1090, 1077, 1041, 1021, 1004, 993,
         987, 918, 904, 882, 860, 827, 770]
y = [(z-32)*(5/9) for z in y] # Convert to Celsius

# Shell measurement position
x = np.arange(5,105,5)

df = pd.DataFrame(index = dti, columns = x)
for i in range(len(y)):
  if i <= 5:
    df[x[i]] = 45*np.sin(t*0.05)+y[i]+np.random.normal(0, 10,len(t))
  elif i <= 15:
    df[x[i]] = 30*np.sin(t*0.01)+y[i]+np.random.normal(0, 7,len(t))
  else:
    df[x[i]] = 35*np.sin(t*0.002)+y[i]+np.random.normal(0, 5,len(t))

df['PV_A'] = np.random.normal(12, 2.3, len(t))
df['PV_B'] = np.random.normal(70, 12, len(t))

descript = pd.DataFrame({5:'KST_5', 10:'KST_10', 15:'KST_15', 20:'KST_20', 25:'KST_25', 30:'KST_30', 35:'KST_35', 40:'KST_40', 45:'KST_45', 50:'KST_50', 55:'KST_55', 60:'KST_60', 
                        65:'KST_65', 70:'KST_70', 75:'KST_75', 80:'KST_80', 85:'KST_85', 90:'KST_90', 95:'KST_96', 100:'KST_100', 'PV_A':'Feed', 'PV_B':'Fuel'}, index =['descript']) 

tags = pd.DataFrame({5:'TZ410.meas', 10:'TZ411.meas', 15:'TZ412.meas', 20:'TZ413.meas', 25:'TZ414.meas', 30:'TZ415.meas', 35:'TZ416.meas', 40:'TZ417.meas', 45:'TZ418.meas', 50:'TZ419.meas', 55:'TZ420.meas', 60:'TZ421.meas', 
                        65:'TZ422.meas', 70:'TZ423.meas', 75:'TZ424.meas', 80:'TZ425.meas', 85:'TZ426.meas', 90:'TZ427.meas', 95:'TZ428.meas', 100:'TZ429.meas', 'PV_A':'TZ101.meas', 'PV_B':'TZ102.meas'}, index =['tags']) 

units = pd.DataFrame({5:'C', 10:'C', 15:'C', 20:'C', 25:'C', 30:'C', 35:'C', 40:'C', 45:'C', 50:'C', 55:'C', 60:'C', 
                        65:'C', 70:'C', 75:'C', 80:'C', 85:'C', 90:'C', 95:'C', 100:'C', 'PV_A':'T/D', 'PV_B':'m3/h'}, index =['units']) 

df = pd.concat([descript,tags,units,df[:]])

Run the following cell to see the format of the synthetic data. The next section describes how to load your own data but it is important to make sure your data (after it is loaded) has the same format as the synthetic data shown below in order for the subsequent code to run. 

In [163]:
df

,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,PV_A,PV_B
descript,KST_5,KST_10,KST_15,KST_20,KST_25,KST_30,KST_35,KST_40,KST_45,KST_50,KST_55,KST_60,KST_65,KST_70,KST_75,KST_80,KST_85,KST_90,KST_96,KST_100,Feed,Fuel
tags,TZ410.meas,TZ411.meas,TZ412.meas,TZ413.meas,TZ414.meas,TZ415.meas,TZ416.meas,TZ417.meas,TZ418.meas,TZ419.meas,TZ420.meas,TZ421.meas,TZ422.meas,TZ423.meas,TZ424.meas,TZ425.meas,TZ426.meas,TZ427.meas,TZ428.meas,TZ429.meas,TZ101.meas,TZ102.meas
units,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,T/D,m3/h
2018-07-01 00:00:00,765.999,748.196,735.835,718.476,666.922,641.633,608.029,587.426,579.773,561.741,549.157,539.923,533.53,530.026,492.044,484.684,472.296,459.918,441.742,409.677,10.3348,70.1475
2018-07-01 01:00:00,766.159,750.401,740.6,721.51,667.334,645.501,608.431,588.467,580.982,561.051,550.022,539.548,533.439,532.049,493.122,484.498,472.606,460.597,441.69,410.517,10.9277,75.4564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-11 03:00:00,787.14,769.101,757.802,738.927,686.709,663.56,613.759,594.128,587.416,567.272,555.863,547.376,539.684,537.501,498.587,490.856,469.886,457.718,439.094,407.882,11.3072,63.2324
2020-10-11 04:00:00,785.399,766.736,757.788,739.193,688.239,663.55,615.141,595.243,587.082,565.989,556.141,547.395,541.284,538.175,498.529,490.477,469.412,457.354,438.959,407.426,14.2209,71.2647
2020-10-11 05:00:00,785.331,767.901,758.181,739.417,685.944,663.063,615.984,595.557,587.693,567.945,557.012,547.698,542.251,537.121,498.787,492.804,469.157,457.167,438.502,406.778,10.1345,70.4792
2020-10-11 06:00:00,786.13,768.307,757.571,740.027,685.032,661.632,616.043,595.985,587.775,568.483,558.301,546.694,540.794,538.346,499.601,490.972,468.454,456.463,437.856,406.32,11.2034,85.1449


## 3. Saving and loading data.



## 4. Interpolating the KST data

In [218]:
d_shell = df.iloc[3:,0:-2]

d_tags = df.iloc[0:3,:]

d_shell = d_shell.T.astype('float')
index = np.arange(x.min(),x.max()+1)
df_int = pd.DataFrame(index=index)
for colname, col in d_shell.iteritems():
  df_int[colname] = np.interp(index, d_shell.index, col)

df_int = df_int.astype(float)

The above code has performed piecewise linear interpolation along the KST profile in unit (e.g. 1 meter) increments. In this case x was equispaced but it does not need to be. The resulting interpolated dataframe is shown below.

In [165]:
df_int

,2018-07-01 00:00:00,2018-07-01 01:00:00,2018-07-01 02:00:00,2018-07-01 03:00:00,2018-07-01 04:00:00,2018-07-01 05:00:00,2018-07-01 06:00:00,2018-07-01 07:00:00,2018-07-01 08:00:00,2018-07-01 09:00:00,2018-07-01 10:00:00,2018-07-01 11:00:00,2018-07-01 12:00:00,2018-07-01 13:00:00,2018-07-01 14:00:00,2018-07-01 15:00:00,2018-07-01 16:00:00,2018-07-01 17:00:00,2018-07-01 18:00:00,2018-07-01 19:00:00,2018-07-01 20:00:00,2018-07-01 21:00:00,2018-07-01 22:00:00,2018-07-01 23:00:00,2018-07-02 00:00:00,2018-07-02 01:00:00,2018-07-02 02:00:00,2018-07-02 03:00:00,2018-07-02 04:00:00,2018-07-02 05:00:00,2018-07-02 06:00:00,2018-07-02 07:00:00,2018-07-02 08:00:00,2018-07-02 09:00:00,2018-07-02 10:00:00,2018-07-02 11:00:00,2018-07-02 12:00:00,2018-07-02 13:00:00,2018-07-02 14:00:00,2018-07-02 15:00:00,...,2020-10-09 16:00:00,2020-10-09 17:00:00,2020-10-09 18:00:00,2020-10-09 19:00:00,2020-10-09 20:00:00,2020-10-09 21:00:00,2020-10-09 22:00:00,2020-10-09 23:00:00,2020-10-10 00:00:00,2020-10-10 01:00:00,2020-10-10 02:00:00,2020-10-10 03:00:00,2020-10-10 04:00:00,2020-10-10 05:00:00,2020-10-10 06:00:00,2020-10-10 07:00:00,2020-10-10 08:00:00,2020-10-10 09:00:00,2020-10-10 10:00:00,2020-10-10 11:00:00,2020-10-10 12:00:00,2020-10-10 13:00:00,2020-10-10 14:00:00,2020-10-10 15:00:00,2020-10-10 16:00:00,2020-10-10 17:00:00,2020-10-10 18:00:00,2020-10-10 19:00:00,2020-10-10 20:00:00,2020-10-10 21:00:00,2020-10-10 22:00:00,2020-10-10 23:00:00,2020-10-11 00:00:00,2020-10-11 01:00:00,2020-10-11 02:00:00,2020-10-11 03:00:00,2020-10-11 04:00:00,2020-10-11 05:00:00,2020-10-11 06:00:00,2020-10-11 07:00:00
5,765.998874,766.159292,767.969245,771.090437,772.820145,775.124725,775.794037,777.824585,778.829481,781.445901,781.565781,783.560339,782.589007,784.497076,785.576104,784.222076,784.346676,785.957805,783.810243,785.078783,783.049311,783.230267,781.556414,779.493431,780.020219,776.870281,777.699530,772.407881,771.368005,769.730238,765.810914,766.274536,763.930758,763.334864,759.697497,758.795191,757.059356,753.012954,752.851939,752.037935,...,749.084766,748.383678,747.070030,746.396171,747.322239,744.981637,745.595661,747.526587,745.815687,747.951257,748.703188,748.951456,750.676532,752.990385,752.713026,754.880357,757.234251,759.402495,760.710712,762.937215,763.436472,765.800039,767.434759,769.170314,770.881379,773.457809,775.370975,778.431259,778.121955,780.016234,782.222345,782.912022,784.393654,783.012284,784.959081,787.139892,785.399331,785.330627,786.130003,785.783077
6,762.438200,763.007587,764.704593,767.857203,769.503902,771.913866,772.216617,774.457804,775.757509,777.966056,778.076661,780.483149,779.616381,781.237044,782.091201,780.711230,781.001251,782.311099,780.374438,781.394645,779.826006,779.752270,778.243149,776.440322,776.288126,773.444575,773.761046,768.847816,768.074221,766.230385,763.183705,763.295070,760.817194,759.458654,756.540666,755.856752,753.331402,749.954845,749.067418,748.563345,...,745.464387,744.859243,743.676980,742.994802,743.394471,741.613537,742.512453,743.693819,742.540161,744.573436,744.911373,745.620483,747.623603,749.278940,749.430895,751.558351,753.687650,755.777075,757.203481,759.437525,760.079139,762.280573,763.991666,765.655079,767.547477,769.991747,771.985090,775.110689,774.803262,776.926869,778.381462,779.386060,780.864759,779.586326,781.555832,783.532027,781.666738,781.844764,782.565430,782.261040
7,758.877525,759.855881,761.439940,764.623968,766.187660,768.703007,768.639198,771.091023,772.685537,774.486211,774.587542,777.405958,776.643755,777.977011,778.606299,777.200385,777.655826,778.664393,776.938633,777.710508,776.602701,776.274273,774.929885,773.387213,772.556033,770.018869,769.822561,765.287751,764.780437,762.730533,760.556495,760.315604,757.703630,755.582443,753.383835,752.918313,749.603449,746.896735,745.282897,745.088755,...,741.844008,741.334808,740.283929,739.593432,739.466704,738.245437,739.429244,739.861051,739.264634,741.195615,741.119558,742.289511,744.570674,745.567496,746.148765,748.236344,7

## 5. Generate interactive spatiotemporal heatmap

In [219]:
def Vis():
    y_slider = widgets.IntSlider(min=d_shell.columns[0].year,max=d_shell.columns[-1].year,value=d_shell.columns[0].year, step=1, description='year')
    m_slider = widgets.IntSlider(min=1,max=12,value=d_shell.columns[0].month, step=1, description='month')
    w_slider = widgets.IntSlider(min=12,max=len(d_shell.T),value=48, step=12, description='window size')
    d_slider = widgets.IntSlider(min=1,max=31,value=1, step=1, description='day')
    h_slider = widgets.IntSlider(min=0,max=24,value=1, step=1, description='hour')

    PV = widgets.Dropdown(options=d_tags.columns.tolist(), description = 'process variable')
    
    ui0 = widgets.VBox([y_slider,m_slider])
    ui1 = widgets.VBox([d_slider,h_slider])
    ui2 = widgets.VBox([w_slider,PV])
    ui3 = widgets.HBox([ui0,ui1,ui2])

    out = widgets.interactive_output(heatmap, {'y_slider':y_slider,'m_slider':m_slider,'d_slider':d_slider,'h_slider':h_slider,'w_slider':w_slider,'PV':PV})
    display(ui3,out)

In [220]:
def heatmap(y_slider, m_slider, d_slider, h_slider, w_slider, PV):
    
    DF = df_int

    # This code accounts for the manual selector for year/month/day/hour
    dt = pd.DataFrame({'year': y_slider,
                   'month': m_slider,
                   'day': d_slider,
                   'hour': h_slider}, index=[0])
    dt = pd.to_datetime(dt)
    # We find the row # of df that matches the value of dt (from the user)
    j = DF.T.index.get_loc(dt.iloc[0])

    SMALL_SIZE = 18     #Font sizes
    MEDIUM_SIZE = 20
    BIGGER_SIZE = 20
    #plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize= MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    
    
    fig, ax = plt.subplots(2,figsize=(30,15))
    plt.style.use("ggplot")
    im = ax[0].imshow(DF.iloc[:,j:j+w_slider],aspect = 'auto', cmap='magma', vmin= d_shell.min().min() - 10, vmax = d_shell.max().max() + 10)
    cbar = ax[0].figure.colorbar(im, ax=ax)#[0])     # Create colorbar
    cbar.set_label(label="Temperature ($^\circ$C)", rotation=-90, va="bottom", fontsize=18)

    # THIS CODE IS FOR THE HEATMAP
    y_m = [z*0.3048 for z in DF.index]
    ax[0].tick_params(axis='both', direction='out')
    ax[0].set_xticks([0,int((w_slider)/4),int((w_slider)/2),int((w_slider)*3/4),w_slider-1])
    ax[0].set_xticklabels([DF.columns[j],DF.columns[j+int((w_slider)/4)],DF.columns[j+int((w_slider)/2)],DF.columns[j+int((w_slider)*3/4)],DF.columns[j+w_slider-1]])    
    ax[0].set_xlabel("Date")
    ax[0].set_yticks([DF.index[0], DF.index[0+int(np.round(len(df_int)/4,0))], DF.index[0+2*int(np.round(len(df_int)/4,0))], DF.index[0+3*int(np.round(len(df_int)/4,0))], DF.index[-1]]) 
    ax[0].set_yticklabels([DF.index[0], DF.index[0+int(np.round(len(df_int)/4,0))], DF.index[0+2*int(np.round(len(df_int)/4,0))], DF.index[0+3*int(np.round(len(df_int)/4,0))], DF.index[-1]])
    ax[0].set_ylabel("Distance from firing end of kiln (m)")
    
    
    ln1, = ax[1].plot(pd.Series(data=df[PV].iloc[3:].values[j:j+w_slider]), label = str(PV) + '\n' + d_tags[PV].loc['tags'])
    #ln1, = ax[1].plot(pd.Series(data=df[d_tags[PV].loc['descript']].values[j:j+w_slider]), label = PV + '\n' + d_tags[PV].loc['tags'])
    ax[1].autoscale(enable=True, axis='x', tight=True)
    ax[1].legend(handles=[ln1],loc="upper right")
    ax[1].set_xlabel("Date")
    ax[1].set_xticks([0,int((w_slider)/4),int((w_slider)/2),int((w_slider)*3/4),w_slider-1])
    ax[1].set_xticklabels([DF.columns[j],DF.columns[j+int((w_slider)/4)],DF.columns[j+int((w_slider)/2)],DF.columns[j+int((w_slider)*3/4)],DF.columns[j+w_slider-1]])
    ax[1].set_ylabel(str(PV) + ' (' + d_tags[PV].loc['units'] + ')')
    
    return plt.show()

In [221]:
Vis()

Output()